## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-01-12-14 +0000,nyt,Who Are the Louisiana Voters Behind a Major Su...,https://www.nytimes.com/2025/10/12/us/politics...
1,2025-10-13-01-08-41 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/12/world/...
2,2025-10-13-01-05-31 +0000,nyt,Why Now? The Lost Chances to Reach a Hostage D...,https://www.nytimes.com/2025/10/12/us/politics...
3,2025-10-13-01-04-52 +0000,48hills,Dear Mark Benioff: Are you a socialist?,https://48hills.org/2025/10/dear-mark-benioff-...
4,2025-10-13-01-03-10 +0000,startribune,Minnesotans welcome Gaza ceasefire but questio...,https://www.startribune.com/gaza-ceasefire-min...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,trump,24
110,china,11
266,hamas,11
13,deal,10
73,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,78
128,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,76
180,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,70
9,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...,67
65,2025-10-12-20-10-31 +0000,nypost,Jake Sullivan begrudgingly credits Trump for I...,https://nypost.com/2025/10/12/us-news/jake-sul...,61


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,78,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...
173,44,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
36,43,2025-10-12-22-22-15 +0000,nypost,Israeli hostage release feels tantalizingly cl...,https://nypost.com/2025/10/12/us-news/hundreds...
78,37,2025-10-12-19-09-17 +0000,nypost,Ian Watkins’ ex shocked pedophile rocker wasn’...,https://nypost.com/2025/10/12/us-news/ian-watk...
82,30,2025-10-12-18-39-55 +0000,nypost,Kamala Harris reveals Biden sent her call to v...,https://nypost.com/2025/10/12/us-news/kamala-h...
9,28,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...
5,26,2025-10-13-01-03-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
32,26,2025-10-12-23-00-00 +0000,wsj,Federal workers are already starting to miss p...,https://www.wsj.com/politics/policy/the-govern...
129,25,2025-10-12-12-33-11 +0000,cbc,Afghanistan says it killed 58 Pakistani soldie...,https://www.cbc.ca/news/world/afghanistan-kill...
58,24,2025-10-12-20-29-41 +0000,nypost,Washington Post roasts Mamdani’s free NYC bus ...,https://nypost.com/2025/10/12/us-news/washingt...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
